# Azure chat completion models with your own data (preview)
この例では、Azure OpenAIサービスのモデルを独自のデータと組み合わせて使用する方法を示します。この機能は現在プレビュー段階です。

Azure OpenAI on your dataを使用すると、モデルの訓練やファインチューニングを行うことなく、GPT-3.5-TurboやGPT-4などのサポートされているチャットモデルを独自のデータ上で実行できます。データ上でモデルを実行することで、より高い精度と速度でデータの上でチャットを行い、分析することが可能になります。Azure OpenAI on your dataの主要な利点の一つは、会話型AIのコンテンツをカスタマイズできることです。モデルが特定のソースにアクセスし、応答をサポートするために参照できるため、回答は事前訓練された知識だけでなく、指定されたデータソースで利用可能な最新情報にも基づいています。この基盤となるデータは、モデルが古い情報や不正確な情報に基づいて応答を生成することを避けるのにも役立ちます。

Azure AI Search（旧Azure Cognitive Search）を使用したAzure OpenAI on your own dataは、知識検索のためのカスタマイズ可能で事前構築されたソリューションを提供し、これを基に会話型AIアプリケーションを構築できます。知識検索とセマンティック検索の代替手法については、[vector databases](https://github.com/openai/openai-cookbook/tree/main/examples/vector_databases)のクックブック例をご確認ください。

## 仕組み

[Azure OpenAI on your own data](https://learn.microsoft.com/azure/ai-services/openai/concepts/use-your-data)は、モデルをあなたのデータと接続し、モデルの出力を向上させる方法でデータを取得・活用する能力を提供します。Azure AI Searchと連携して、ユーザーの入力と提供された会話履歴に基づいて、指定されたデータソースからデータが取得されます。そのデータは拡張され、プロンプトとしてモデルに再送信されることで、モデルが応答を生成する際に使用できるコンテキスト情報を提供します。

詳細については、[Azure OpenAI Serviceのデータ、プライバシー、セキュリティ](https://learn.microsoft.com/legal/cognitive-services/openai/data-privacy?context=%2Fazure%2Fai-services%2Fopenai%2Fcontext%2Fcontext)をご覧ください。

## 前提条件
開始するにあたり、いくつかの前提条件について説明します。

Azure OpenAI Serviceに適切にアクセスするには、[Azure Portal](https://portal.azure.com)で適切なリソースを作成する必要があります（詳細な手順については[Microsoft Docs](https://learn.microsoft.com/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal)の詳細ガイドをご確認ください）。

Azure OpenAIモデルで独自のデータを使用するには、以下が必要です：

1. Azure OpenAIへのアクセスと、チャットモデルがデプロイされたリソース（例：GPT-3またはGPT-4）
2. Azure AI Search（旧Azure Cognitive Search）リソース
3. Azure Blob Storageリソース
4. データとして使用するドキュメント（[データソースオプション](https://learn.microsoft.com/azure/ai-services/openai/concepts/use-your-data#data-source-options)を参照）

ドキュメントをBlob Storageにアップロードし、Azure AI Studioを使用してインデックスを作成する方法の完全なウォークスルーについては、この[クイックスタート](https://learn.microsoft.com/azure/ai-services/openai/use-your-data-quickstart?pivots=programming-language-studio&tabs=command-line)をご覧ください。

## セットアップ

まず、必要な依存関係をインストールします。

In [ ]:
! pip install "openai>=1.0.0,<2.0.0"
! pip install python-dotenv

この例では、`dotenv`を使用して環境変数を読み込みます。Azure OpenAIとSearchインデックスに接続するために、以下の変数を`KEY=VALUE`形式で`.env`ファイルに追加する必要があります：

* `AZURE_OPENAI_ENDPOINT` - Azure OpenAIエンドポイント。これはAzure PortalのAzure OpenAIリソースの「キーとエンドポイント」で確認できます。
* `AZURE_OPENAI_API_KEY` - Azure OpenAI APIキー。これはAzure PortalのAzure OpenAIリソースの「キーとエンドポイント」で確認できます。Azure Active Directory認証を使用する場合は省略してください（下記の`Microsoft Active Directoryを使用した認証`を参照）。
* `SEARCH_ENDPOINT` - AI Searchエンドポイント。このURLはAzure PortalのSearchリソースの「概要」で確認できます。
* `SEARCH_KEY` - AI Search APIキー。Azure PortalのSearchリソースの「キー」で確認できます。
* `SEARCH_INDEX_NAME` - 独自データで作成したインデックスの名前。

In [ ]:
import os
import openai
import dotenv

dotenv.load_dotenv()

### 認証

Azure OpenAI サービスは、APIキーとAzure Active Directoryトークン認証情報を含む複数の認証メカニズムをサポートしています。

In [2]:
use_azure_active_directory = False  # Set this flag to True if you are using Azure Active Directory

#### APIキーを使用した認証

OpenAI SDKを*Azure APIキー*を使用するように設定するには、`api_key`をエンドポイントに関連付けられたキーに設定する必要があります（このキーは[Azure Portal](https://portal.azure.com)の*「リソース管理」*の下にある*「キーとエンドポイント」*で確認できます）。ここでリソースのエンドポイントも確認できます。

In [3]:
if not use_azure_active_directory:
    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
    api_key = os.environ["AZURE_OPENAI_API_KEY"]
    # set the deployment name for the model we want to use
    deployment = "<deployment-id-of-the-model-to-use>"

    client = openai.AzureOpenAI(
        base_url=f"{endpoint}/openai/deployments/{deployment}/extensions",
        api_key=api_key,
        api_version="2023-09-01-preview"
    )

#### Azure Active Directoryを使用した認証
次に、Azure Active Directoryを介して認証する方法を見てみましょう。まず、`azure-identity`ライブラリをインストールします。このライブラリは、認証に必要なトークン認証情報を提供し、`get_bearer_token_provider`ヘルパー関数を通じてトークン認証情報プロバイダーの構築を支援します。静的トークンを`AzureOpenAI`に提供するよりも`get_bearer_token_provider`を使用することが推奨されます。なぜなら、このAPIは自動的にトークンをキャッシュし、更新してくれるからです。

Azure OpenAIでAzure Active Directory認証を設定する方法の詳細については、[ドキュメント](https://learn.microsoft.com/azure/ai-services/openai/how-to/managed-identity)を参照してください。

In [ ]:
! pip install "azure-identity>=1.15.0"

In [5]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

if use_azure_active_directory:
    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
    api_key = os.environ["AZURE_OPENAI_API_KEY"]
    # set the deployment name for the model we want to use
    deployment = "<deployment-id-of-the-model-to-use>"

    client = openai.AzureOpenAI(
        base_url=f"{endpoint}/openai/deployments/{deployment}/extensions",
        azure_ad_token_provider=get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"),
        api_version="2023-09-01-preview"
    )

> 注意: AzureOpenAIは、以下の引数が提供されていない場合、対応する環境変数から推論します：

- `api_key` は `AZURE_OPENAI_API_KEY` から
- `azure_ad_token` は `AZURE_OPENAI_AD_TOKEN` から
- `api_version` は `OPENAI_API_VERSION` から
- `azure_endpoint` は `AZURE_OPENAI_ENDPOINT` から

## 独自データを使用したチャット補完モデル

### コンテキストの設定

この例では、モデルがAzure AIサービスのドキュメントデータに基づいて応答するようにしたいと考えています。前述の[クイックスタート](https://learn.microsoft.com/azure/ai-services/openai/use-your-data-quickstart?tabs=command-line&pivots=programming-language-studio)に従って、[Azure AIサービスと機械学習](https://learn.microsoft.com/azure/ai-services/cognitive-services-and-machine-learning)ドキュメントページの[markdown](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/ai-services/cognitive-services-and-machine-learning.md)ファイルを検索インデックスに追加しました。これで、モデルはAzure AIサービスと機械学習に関する質問に答える準備が整いました。

### コード

これで、Chat Completionsで独自のデータを使用したAzureを利用できるようになりました。`dataSources`に検索エンドポイント、キー、インデックス名を提供することで、モデルに投げかけられた質問は独自のデータに基づいて回答されるようになります。レスポンスには追加のプロパティ`context`が提供され、モデルが質問に答えるために参照したデータが表示されます。

In [ ]:
completion = client.chat.completions.create(
    messages=[{"role": "user", "content": "What are the differences between Azure Machine Learning and Azure AI services?"}],
    model=deployment,
    extra_body={
        "dataSources": [
            {
                "type": "AzureCognitiveSearch",
                "parameters": {
                    "endpoint": os.environ["SEARCH_ENDPOINT"],
                    "key": os.environ["SEARCH_KEY"],
                    "indexName": os.environ["SEARCH_INDEX_NAME"],
                }
            }
        ]
    }
)
print(f"{completion.choices[0].message.role}: {completion.choices[0].message.content}")

# `context` is in the model_extra for Azure
print(f"\nContext: {completion.choices[0].message.model_extra['context']['messages'][0]['content']}")

モデルからのレスポンスをストリーミングしたい場合は、`stream=True`キーワード引数を渡すことができます：

In [ ]:
response = client.chat.completions.create(
    messages=[{"role": "user", "content": "What are the differences between Azure Machine Learning and Azure AI services?"}],
    model=deployment,
    extra_body={
        "dataSources": [
            {
                "type": "AzureCognitiveSearch",
                "parameters": {
                    "endpoint": os.environ["SEARCH_ENDPOINT"],
                    "key": os.environ["SEARCH_KEY"],
                    "indexName": os.environ["SEARCH_INDEX_NAME"],
                }
            }
        ]
    },
    stream=True,
)

for chunk in response:
    delta = chunk.choices[0].delta

    if delta.role:
        print("\n"+ delta.role + ": ", end="", flush=True)
    if delta.content:
        print(delta.content, end="", flush=True)
    if delta.model_extra.get("context"):
        print(f"Context: {delta.model_extra['context']}", end="", flush=True)